# Module 8 - Programming Assignment


In [86]:
from copy import deepcopy
from collections import Counter
import math

## Decision Trees

For this assignment you will be implementing and evaluating a Decision Tree using the ID3 Algorithm (**no** pruning or normalized information gain). Use the provided pseudocode. The data is located at (copy link):

http://archive.ics.uci.edu/ml/datasets/Mushroom

**Just in case** the UCI repository is down, which happens from time to time, I have included the data and name files on Blackboard.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        No Pandas. The only acceptable libraries in this class are those contained in the `environment.yml`. No OOP, either. You can used Dicts, NamedTuples, etc. as your abstract data type (ADT) for the the tree and nodes.
    </p>
</div>

One of the things we did not talk about in the lectures was how to deal with missing values. There are two aspects of the problem here. What do we do with missing values in the training data? What do we do with missing values when doing classifcation?

There are a lot of different ways that we can handle this.
A common algorithm is to use something like kNN to impute the missing values.
We can use conditional probability as well.
There are also clever modifications to the Decision Tree algorithm itself that one can make.

We're going to do something simpler, given the size of the data set: remove the observations with missing values ("?").

You must implement the following functions:

`train` takes training_data and returns the Decision Tree as a data structure.

```
def train(training_data):
   # returns the Decision Tree.
```

`classify` takes a tree produced from the function above and applies it to labeled data (like the test set) or unlabeled data (like some new data).

```
def classify(tree, observations, labeled=True):
    # returns a list of classifications
```

`evaluate` takes a data set with labels (like the training set or test set) and the classification result and calculates the classification error rate:

$$error\_rate=\frac{errors}{n}$$

Do not use anything else as evaluation metric or the submission will be deemed incomplete, ie, an "F". (Hint: accuracy rate is not the error rate!).

`cross_validate` takes the data and uses 10 fold cross validation (from Module 3!) to `train`, `classify`, and `evaluate`. **Remember to shuffle your data before you create your folds**. I leave the exact signature of `cross_validate` to you but you should write it so that you can use it with *any* `classify` function of the same form (using higher order functions and partial application).

Following Module 3's assignment, `cross_validate` should print out a table in exactly the same format. What you are looking for here is a consistent evaluation metric cross the folds. Print the error rate to 4 decimal places. **Do not convert to a percentage.**

```
def pretty_print_tree(tree):
    # pretty prints the tree
```

This should be a text representation of a decision tree trained on the entire data set (no train/test).

To summarize...

Apply the Decision Tree algorithm to the Mushroom data set using 10 fold cross validation and the error rate as the evaluation metric. When you are done, apply the Decision Tree algorithm to the entire data set and print out the resulting tree.

**Note** Because this assignment has a natural recursive implementation, you should consider using `deepcopy` at the appropriate places.

-----

## Load the Data

The function `parse_data` loads the data from the specified file and returns a List of Lists. The outer List is the data set and each element (List) is a specific observation. Each value of an observation is for a particular measurement. This is what we mean by "tidy" data.

The function also returns the *shuffled* data because the data might have been collected in a particular order that *might* bias training.

In [87]:
import random
from typing import List, Dict, Tuple, Callable

In [88]:
def parse_data(file_name: str) -> List[List]:
    data = []
    file = open(file_name, "r")
    for line in file:
        datum = [value for value in line.rstrip().split(",")]
        data.append(datum)
    random.shuffle(data)
    return data

In [89]:
data = parse_data("agaricus-lepiota.data")

In [90]:
len(data)

8124

## Train/Test Splits - n folds

With n fold cross validation, we divide our data set into n subgroups called "folds" and then use those folds for training and testing. You pick n based on the size of your data set. If you have a small data set--100 observations--and you used n=10, each fold would only have 10 observations. That's probably too small. You want at least 30. At the other extreme, we generally don't use n > 10.

With 8124 observations, n = 10 is fine so we will have 10 folds.
`create_folds` will take a list (xs) and split it into `n` equal folds with each fold containing one-tenth of the observations.

In [91]:
def create_folds(xs: List, n: int) -> List[List[List]]:
    k, m = divmod(len(xs), n)
    # be careful of generators...
    return list(xs[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [92]:
folds = create_folds(data, 10)

In [93]:
len(folds)

10

In [94]:
def create_train_test(folds: List[List[List]], index: int) -> Tuple[List[List], List[List]]:
    training = []
    test = []
    for i, fold in enumerate(folds):
        if i == index:
            test = fold
        else:
            training = training + fold
    return training, test

In [95]:
train, test = create_train_test(folds, 0)

In [96]:
len(train)

7311

In [97]:
len(test)

813

<a id="homogeneous"></a>
## homogeneous

The homogeneous function checks if all the sub-lists have the same class. **Used by**: [id3](#id3)

* **data List[List[str]]:** A collection of sub-lists containing the data of the problem
* **Returns: bool:** Returns True if all sub-lists same class, otherwise False

In [98]:
def homogeneous(data):
    first = data[0][0]
    return all([first == instance[0] for instance in data])

In [99]:
assert homogeneous([['p','x'], ['p','x'], ['p','x']]) == True
assert homogeneous([['p','x'], ['e','x'], ['p','x']]) == False
assert homogeneous([['p','y'], ['p','y'], ['p','y']]) == True

<a id="majority_label"></a>
## majority_label

The majority_label function measure the class label occurinng most frequently in the data. **Used by**: [id3](#id3)

* **data List[List[str]]:** A collection of sub-lists containing the data of the problem
* **Returns: str:** Returns str of class label

In [100]:
def majority_label(data):
    list_of_first = [row[0] for row in data]
    counts = Counter(list_of_first)
    return counts.most_common(1)[0][0]

In [101]:
data1 = [['p','x'], ['p','x'], ['p','x']]
data2 = [['p','x'], ['e','x'], ['p','x']]
data3 = [['e','x'], ['e','x'], ['e','x']]
data4 = [['p','x'], ['p','x'], ['e','x'], ['e','x']]
assert majority_label(data1) == 'p'
assert majority_label(data2) == 'p'
assert majority_label(data3) == 'e'
assert majority_label(data4) == 'p'


In [102]:
atrib = {'cap-shape':1, 
         'cap-surface':2, 
         'cap-color':3 , 
         'bruises?':4, 
         'odor':5, 
         'gill-attachment':6 , 
         'gill-spacing':7 , 
         'gill-size':8, 
         'gill-color':9, 
         'stalk-shape':10, 
         'stalk-root':11, 
         'stalk-surface-above-ring':12, 
         'stalk-surface-below-ring':13, 
         'stalk-color-above-ring':14, 
         'stalk-color-below-ring':15, 
         'veil-type':16, 
         'veil-color':17, 
         'ring-number':18, 
         'ring-type':19, 
         'spore-print-color':20, 
         'population':21, 
         'habitat':22}

<a id="entropy"></a>
## entropy
computes the entropy of a set of classes or labels. Entropy measures the uncertainty of a set
 **Used by**: [pick_best_attribute](#pick_best_attribute)

* **classes List[str]:** a list of classes
* **Returns: float** entropy value

In [103]:
def entropy(classes):
    label_counts = Counter(classes)
    entropy = 0.0
    total = len(classes)
    for count in label_counts.values():
        prob = count / total
        entropy -= prob * math.log2(prob)
    return entropy

In [104]:
assert math.isclose(entropy([1, 1, 0, 0,]), 1, abs_tol=1e-2)
assert math.isclose(entropy([1, 0, 0, 0,]), .81, abs_tol=1e-2)
assert math.isclose(entropy([0, 0, 0, 0,]), 0, abs_tol=1e-5)

<a id="attribute_domain"></a>
## attribute_domain
This function computes the frequency of each unique value for a specified attribute in a dataset. **Used by**: [pick_best_attribute](#pick_best_attribute)

* **data List[List[str]]:** A collection of sub-lists containing the data of the problem
* **attribute int:** column number in the data for which we want to compute the frequency of its values.
* **Returns:** A Counter object

In [105]:
def attribute_domain(data, attribute):
    return Counter([row[attribute] for row in data])

In [106]:
assert attribute_domain([['a'], ['a'],['b'],['c']], 0) == {'a':2, 'b':1,'c':1}
assert attribute_domain([['a'], ['a'],['a'],['c']], 0) == {'a':3, 'c':1}
assert attribute_domain([['a'], ['a'],['a'],['a']], 0) == {'a':4}

<a id="filter_data"></a>
## filter_data
This function computes the frequency of each unique value for a specified attribute in a dataset. **Used by**: [pick_best_attribute](#pick_best_attribute)

* **data List[List[str]]:** A collection of sub-lists containing the data of the problem
* **column_index int:** column number to perfom match on
* **category str:** category value to match with
* **Returns:** List[List[str]] cleaned data that only contains the category value

In [107]:
def filter_data(data, column_index, category):
    return [row for row in data if row[column_index] == category]

In [108]:
assert  filter_data([['a'], ['a'],['b'],['c']], 0, 'a') == [['a'],['a']]
assert  filter_data([['a'], ['a'],['b'],['c']], 0, 'b') == [['b']]
assert  filter_data([['a'], ['a'],['b'],['c']], 0, 'd') == []

In [109]:
def pick_best_attribute(data, attributes):
    results_k=[]
    results_i=[]
    starting_entropy = entropy([row[0] for row in data])
    for attribute, value in zip(attributes,attributes.values()):
        for k in attribute_domain(data, attribute).keys():
            new_data= filter_data(data, attribute, k)
            results_k.append(value*entropy([row[0] for row in new_data]))
        results_i.append((sum(results_k),attribute))       
    return sorted(results_i)[-1]


In [110]:
def pretty_print_tree(tree):
    # pretty prints the tree
    pass

In [ ]:
def id3( data, attributes, default)
    if len(data) == 0
        return None
    if homogeneous(data)
        return data[0][0] # return class label
    if len(attributes) == 0
        return majority_label(data)
    best_attr = pick_best_attribute(data, attributes)[1]
    node = new Node( best_attribute) ### needs work ---want to use a dict of dict
    default_label = majority_label(data)
    for value in attribute_domain(data, attribute).keys()
        subset = filter_data(data, best_attr, value)
        child = id3( subset, attributes - best_attr, default_label)
        add child to node # needs work ---want to use a dict of dict
    return node

In [ ]:
def train(training_data):
    # returns the Decision Tree
    pass

In [ ]:
def classify(tree, observations, labeled=True):
    # returns a list of classifications
    pass

In [ ]:
 def evaluate(Training_set_or_test_set, classifcations_results):
#         calculates the classification error rate:
    pass

In [ ]:
def cross_validate(data):
#     uses 10 fold cross validation (from Module 3!) to train, classify, and evaluate.
    T= train(data) 
    c= classifiy(T, data, labeled=True)
    e= evaluate(data, c)
    pass

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.